In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

Cloning into 'yolov5'...
remote: Enumerating objects: 16575, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 16575 (delta 28), reused 37 (delta 18), pack-reused 16522
Receiving objects: 100% (16575/16575), 15.09 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (11381/11381), done.
/content/yolov5
HEAD is now at 064365d8 Update parse_opt() in export.py to work as in train.py (#10789)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
Setup complete. Using torch 2.2.1+cu121 CPU


In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
!python detect.py --weights /content/best1.pt --img 640 --conf 0.2 --source /content/vid5.mp4 --save-txt

detect: weights=['/content/best1.pt'], source=/content/vid5.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-72-g064365d8 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
video 1/1 (1/1944) /content/vid5.mp4: 480x640 5 dropletss, 374.4ms
video 1/1 (2/1944) /content/vid5.mp4: 480x640 5 dropletss, 367.7ms
video 1/1 (3/1944) /content/vid5.mp4: 480x640 5 dropletss, 368.7ms
video 1/1 (4/1944) /content/vid5.mp4: 480x640 5 dropletss, 357.0ms
video 1/1 (5/1944) /content/vid5.mp4: 480x640 5 dropletss, 342.9ms
video 1/1 (6/1944) /content/vid5.mp4: 480x640 5 droplets

In [ ]:
# Generation speed

import os
import numpy as np

def parse_droplet_positions(file_path):
    positions = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            # Assuming x, y coordinates are the 2nd and 3rd items
            center_x = float(parts[1])
            center_y = float(parts[2])
            positions.append((center_x, center_y))
    return positions

def is_unique_droplet(new_droplet, existing_droplets, threshold=0.005):
    for droplet in existing_droplets:
        if np.linalg.norm(np.array(droplet) - np.array(new_droplet)) < threshold:
            return False
    return True

def process_droplets(directory):
    unique_droplets = []
    # Using a more robust way to handle filenames that contain numbers
    for filename in sorted(os.listdir(directory), key=lambda x: int(x.split('_')[-1].split('.')[0])):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            positions = parse_droplet_positions(file_path)
            for pos in positions:
                if is_unique_droplet(pos, unique_droplets):
                    unique_droplets.append(pos)
    return len(unique_droplets)

def calculate_generation_speed(total_frames, fps_actual, total_unique_droplets):
    time_of_recording = total_frames / fps_actual
    generation_speed = total_unique_droplets / time_of_recording
    return generation_speed

# Constants and inputs
directory = 'runs/detect/exp/labels'
total_frames = int(input("Enter the total number of frames in the video: "))
fps_actual = float(input("Enter the original frames per second (FPS) during recording: "))

# Processing
total_unique_droplets = process_droplets(directory)
generation_speed = calculate_generation_speed(total_frames, fps_actual, total_unique_droplets)

print(f"Total Unique Droplets: {total_unique_droplets}")
print(f"Generation Speed: {generation_speed:.2f} droplets/sec")


Enter the total number of frames in the video: 1944
Enter the original frames per second (FPS) during recording: 7000
Total Unique Droplets: 73
Generation Speed: 262.86 droplets/sec


In [ ]:
#Volume Calculation using xyxy

import math
import numpy as np

def calculate_volume(bbox, px_per_micron, chamber_height, is_oval):
    """
    Calculate the volume of a droplet based on its bounding box, resolution, and chamber height.

    Parameters:
    - bbox: tuple (xmin, ymin, xmax, ymax) bounding box coordinates
    - px_per_micron: float, number of pixels per micron
    - chamber_height: float, height of the chamber in micrometers
    - is_oval: boolean, whether the droplet is oval-shaped

    Returns:
    - volume in picolitres
    """
    xmin, ymin, xmax, ymax = bbox
    width_px = xmax - xmin
    height_px = ymax - ymin

    # Convert pixel dimensions to micrometers
    width_um = width_px / px_per_micron
    height_um = height_px / px_per_micron

    if is_oval:
        # Use the oval formula for volume calculation
        L = max(width_um, height_um)  # Length of the drop
        W = min(width_um, height_um)  # Width of the drop
        H = chamber_height
        c = 0.5
        volume = ((H * W) - (4 - math.pi) * ((2 / H + 2 / W)**-2) - (c * H**2)) * ((L - (W / 3))) / 1000
    else:
        # Calculate the diameter of the droplet
        diameter_um = min(width_um, height_um)  # Assuming the smaller dimension as the diameter
        drop_radius = diameter_um / 2

        if diameter_um > chamber_height:
            # Use the specific formula for volume when diameter is greater than chamber height
            volume = (2 * math.pi * chamber_height * (drop_radius - chamber_height)**2 +
                      (math.pi**2 * chamber_height**2 * (drop_radius - chamber_height * (1 - 4 / (3 * math.pi)))) / 1000)
        else:
            # Assume spherical droplet if not specified otherwise
            volume = 0

    return volume

volume = calculate_volume(bbox, px_per_micron, chamber_height, is_oval)
print(f"The volume of the droplet is {volume:.2f} picolitres")


In [ ]:
import os
import math

def calculate_volume(width_um, height_um, chamber_height, is_oval):
    if is_oval:
        # Use the oval formula for volume calculation
        L = max(width_um, height_um)  # Length of the drop
        W = min(width_um, height_um)  # Width of the drop
        H = chamber_height
        c = 0.5
        volume = ((H * W) - (4 - math.pi) * ((2 / H + 2 / W)**-2) - (c * H**2)) * ((L - (W / 3))) / 1000
        drop_radius = 0
    else:
        # Calculate the diameter of the droplet
        diameter_um = min(width_um, height_um)
        drop_radius = diameter_um / 2

       # if diameter_um > chamber_height:
            # Use the specific formula for volume when diameter is greater than chamber height
        volume = (2 * math.pi * chamber_height * (drop_radius - chamber_height)**2 +
                      (math.pi**2 * chamber_height**2 * (drop_radius - chamber_height * (1 - 4 / (3 * math.pi)))) / 1000)
       # else:
            # Assume spherical droplet if not specified otherwise
           # volume = 0

    return volume , drop_radius

def convert_to_micrometers(norm_width, norm_height, img_width, img_height, px_per_micron):
    width_um = norm_width * img_width / px_per_micron
    height_um = norm_height * img_height / px_per_micron
    return width_um, height_um

def process_droplets(directory, img_width, img_height, px_per_micron, chamber_height, is_oval):
    volumes = []  # List to accumulate volumes for average calculation
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    parts = line.strip().split()
                    norm_width = float(parts[3])
                    norm_height = float(parts[4])

                    width_um, height_um = convert_to_micrometers(norm_width, norm_height, img_width, img_height, px_per_micron)
                    volume, drop_radius = calculate_volume(width_um, height_um, chamber_height, is_oval)
                    volumes.append(volume)
                    print(f"File: {filename}, Droplet Volume: {volume:.2f} picolitres, Radius = {drop_radius:.2f} microns")

    # Calculate and print the mean volume if there are any volumes recorded
    if volumes:
        mean_volume = sum(volumes) / len(volumes)
        print(f"Mean Volume of All Droplets: {mean_volume:.2f} picolitres")
    else:
        print("No volumes to calculate mean.")

# Constants and parameters
directory = 'runs/detect/exp3/labels'
img_width = 480 # Image width in pixels
img_height = 640 # Image height in pixels
px_per_micron = 0.3048700048 #0.32800865426 #3.048700048  # Correct Pixel to micron conversion factor
chamber_height = 24  # Chamber height in micrometers
is_oval = False  # Whether the droplet is oval-shaped

process_droplets(directory, img_width, img_height, px_per_micron, chamber_height, is_oval)


File: vid4_483.txt, Droplet Volume: 1884.67 picolitres, Radius = 20.50 microns
File: vid4_483.txt, Droplet Volume: 366.47 picolitres, Radius = 22.55 microns
File: vid4_483.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_483.txt, Droplet Volume: 3416.76 picolitres, Radius = 28.70 microns
File: vid4_483.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_483.txt, Droplet Volume: 3416.76 picolitres, Radius = 28.70 microns
File: vid4_483.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_353.txt, Droplet Volume: 14044.41 picolitres, Radius = 14.35 microns
File: vid4_353.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_353.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_353.txt, Droplet Volume: 1132.53 picolitres, Radius = 26.65 microns
File: vid4_353.txt, Droplet Volume: 3416.76 picolitres, Radius = 28.70 microns
File: vid4_353.txt, Droplet Volume: 115.72 picolitre